## 2. Masking AKA Clipping with Rasterio

Okay--I admit this is slightly complicated, but doing GIS in the wild can be that way sometimes!  

There's a bunch of ways you may want to clip something, so consider this a primer on what it really takes to clip a raster.  
Essentially, you need:
1. An input image
2. Something you want to clip it with, in this case, we'll use geometries from a shapefile
3. You need the geometry of the clipping object as a [geojson-like dict](https://rasterio.readthedocs.io/en/stable/api/rasterio.mask.html)
4. And of course you need all of the critical descriptive information to go along with these such as the CRS, the transformation, the width, the height, the color profile...  

Let's do it!

In [1]:
import rasterio as rio
from rasterio.plot import show
from rasterio.mask import mask #this is the masking aka clip module
import matplotlib.pyplot as plt
import geopandas as gpd #to read the input shapefile
import os

Important: only run this once or you'll wind up in the wrong directory.
```python
os.chdir('../')
os.getcwd()
```

First, let's read in the image:
```python
src = rio.open('workshopdata/NAIP_Campus.tif')
```

Let's take a peek:
```python
show(src.read(1))
```

Let's clip so that we're only working with the quad area...

Read in the campus shapefile:
```python
shape = gpd.read_file('workshopdata/UCB_MainCampus_Boundaries.shp')
```

Let's take a look:
```python
shape.plot(figsize = (12,12))
```

Okay, but if we're going to clip with this, it needs to be in the same CRS as the image, this is easy to change using geopandas:
```python
src.crs
```

Change it to EPSG 26913
```python
shape = shape.to_crs(26913)
```

Now, let's get the geometry of the southwest part of main campus:


We'll use GeoPandas to access the geometry of that particular feature. 
```python
swQuad = shape.geometry[22]
```

### Okay... 

From the Rasterio docs: The values must be a GeoJSON-like dict or an object that implements the Python geo interface protocol (such as a [Shapely](https://shapely.readthedocs.io/en/stable/geometry.html) Polygon). 

Fortunately, that's what we have:
```python
print(swQuad)
```

Perfect!  

Now finally, we can run the mask function to get the clipped image. mask() will return two things: the image, and it's affine transformation. You must set crop to True--without it, the image will be the same size but with no values beyond our clip area. Setting crop to True gets rid of those areas:
```python
out_img, out_transform = mask(src, [swQuad], crop=True)
```
** One other caveat: wrap the swQuad variable in square brackets so rasterio knows to treat it like an iterable.

Let's take a look...
```python
show(out_img)
```

### Cool! 

### ...but...

Okay, maybe you don't want it clipped to the exact geometry... maybe you would rather have it by the extent of the geometry?

Okay, let's do it!

What we'll need is the bounding box geometry of our swQuad shape. We can use geopandas' envelope property:
```python
envelope = swQuad.envelope
envelope
```

print it...

#### Siiiick!

Now, let's do it again, this time with the envelope as the geometry object:
```python
out_img, out_transform = mask(src, [envelope], crop=True)
```

Check it out:
```python
show(out_img)
```

### Awesome. If the color looks funny, we'll sort that out when we save it... 

Now, let's get the original profile
```python
src.profile
```

Copy it to a variable:
```python
profile = src.profile.copy()
```
(Note, this is the same as the kwargs variable we used in the last notebook)

Grab the height and the width of the output image:
```python
height = out_img.shape[1]
width = out_img.shape[2]
```

Update the profile. Since we have a new transform, a new width & height, we need to use the `.update` method to take the old profile and update certain elements:
```python
profile.update(transform = out_transform, 
               width = width, 
               height = height, 
               photometric = 'rgb', 
               alpha = 'no')
```
(more on photometric & alpha in the next notebook)

Open a new empty dataset:
```python
new = rio.open('NAIP_swQuad.tif', 'w', **profile)
```

Write it to disc:
```python
new.write(out_img)
```

Close it...
```python
new.close()
```

#### Done!  